## Рассчет экономической эффективности.
### Методика расчета:

Для анализа загружаются данные за три года (текущий, на который приходится сформированная рекомендация и два до него для анализа тренда по выручке и кол-ву проданных билетов)
Загруженные данные ключают как продажи АФЛ, так и конкурентов

Источники данных: DDS, CDD
CDD - Источник данных по продажам АФЛ, содежаций следующие поля:
    1. Код тарифа
    2. Дата продажи
    3. Дата перелета
    
DDS   


Дополнительные сведения:
1. АФЛ, принимая рекомендацию может изменить набор тарифов, периодов или же, приняв рекомендацию, добавить сейловый тариф, вместо непосредственного принятия рекомендации.

#### Подход 1
Сравниваются данные по выручкам и проданным билетам на направлении рекомендции до приняти и после. Также производится сопоставление данных года приняти рекомендации с данными за предыдущие два года.

Диапазон оценивания: месяц после принятия рекомендации
Данные:
    1. Продажи АФЛ за три года, включая год принятия рекомендации на анализируемом направлении
    2. Продажи конкурентов за три года, включая год принятия рекомендации на анализируемом направлении

1. Для АФЛ строится  график с соотношением выручки до и после месяца рекомедации
2. Для АФЛ строится  график с соотношением продаж до и после месяца рекомедации
3. Для конкурентов строится график с соотношением продаж до и после месяца рекомедации
4. Делаются выводы об относительном поведении 

In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [134]:
# 1. SU data loading
su_data = pd.read_csv('data/SU_PARBKK_DATA.csv', sep=';', parse_dates=['SALE_DATE', 'FLIGHT_MONTH'])
su_data['SALE_YEAR'] = pd.DatetimeIndex(su_data['SALE_DATE']).year
su_data['MONTH_NUM'] = pd.DatetimeIndex(su_data['FLIGHT_MONTH']).month
su_data.head()

,SALE_DATE,FLIGHT_MONTH,TOTAL_AMT,TOTAL_DOCS,SALE_YEAR,MONTH_NUM
0,2013-06-07,2013-07-01,1361.682172,2,2013,7
1,2014-03-04,2014-03-01,507.344872,1,2014,3
2,2014-03-04,2014-07-01,617.876028,4,2014,7
3,2014-03-04,2014-08-01,0.000000,2,2014,8
4,2016-09-07,2016-10-01,2185.044994,4,2016,10


In [314]:
# Analysis function
# props: su_data, sale date (as DD-YY), analysing month num
# return: pandas df with columns [YEAR, SALES_BEFORE_DATE, SALES_AFTER_DATE, DOCS_BEFORE_DATE, DOCS_AFTER_DATE]
def process_su_data(su_data, sale_date, month_num):
    _loc_su_data = su_data[su_data['MONTH_NUM'] == month_num].copy()
    dataset = []
    for year in su_data['SALE_YEAR'].unique():
        year_sale_date = sale_date + '-' + str(year)
        year_rec = []
        year_rec.append(year)
        year_rec.append(month_num)
        year_rec.append(_loc_su_data[(_loc_su_data['SALE_YEAR'] == year)&
                                     (_loc_su_data['SALE_DATE'] <= year_sale_date)]['TOTAL_AMT'].sum())
        
        year_rec.append(_loc_su_data[(_loc_su_data['SALE_YEAR'] == year)&
                                     (_loc_su_data['SALE_DATE'] > year_sale_date)]['TOTAL_AMT'].sum())
        
        year_rec.append(_loc_su_data[(_loc_su_data['SALE_YEAR'] == year)&
                                     (_loc_su_data['SALE_DATE'] <= year_sale_date)]['TOTAL_DOCS'].sum())
        
        year_rec.append(_loc_su_data[(_loc_su_data['SALE_YEAR'] == year)&
                                     (_loc_su_data['SALE_DATE'] > year_sale_date)]['TOTAL_DOCS'].sum())
        
        dataset.append(year_rec)
        
    data_df = pd.DataFrame(dataset, columns=['YEAR', 'MONTH','SALES_BEFORE_DATE', 
                                          'SALES_AFTER_DATE', 'DOCS_BEFORE_DATE', 
                                          'DOCS_AFTER_DATE'])
    
    data_df['SALES_DIFFERENCE'] = data_df['SALES_AFTER_DATE'] - data_df['SALES_BEFORE_DATE']
    data_df['DOCS_DIFFERENCE'] = data_df['DOCS_AFTER_DATE'] - data_df['DOCS_BEFORE_DATE']
    
    
    data_df.sort_values(by='YEAR', inplace=True)    
    return data_df

def plot_sales(su_data, rec_year, data_column = 'SALES_DIFFERENCE'):
    rec_year_diff = su_data[su_data.YEAR == rec_year][data_column].values[0] - \
                su_data[su_data.YEAR == rec_year - 1][data_column].values[0]
    
    #rec_year_diff = rec_year_diff.values[0]
    month = su_data['MONTH'].values[0]
    
    
    
    
    if data_column == 'SALES_DIFFERENCE':
        perc_diff = 1 - su_data[su_data.YEAR == rec_year]['SALES_BEFORE_DATE']/su_data[su_data.YEAR == rec_year]['SALES_AFTER_DATE']
        perc_diff = perc_diff.values[0]
        perc_diff_prev = 1 - su_data[su_data.YEAR == rec_year - 1]['SALES_BEFORE_DATE']/su_data[su_data.YEAR == rec_year - 1]['SALES_AFTER_DATE']
        perc_diff_prev = perc_diff_prev.values[0]
        title = "В {}-м месяце {} года после применения рекомендации <br>кол-во проданных билетов {} на {:.2f}$ ({:.2f}%)<br>"\
             "по сравнению с предыдущим годом"
    else:
        perc_diff = 1 - su_data[su_data.YEAR == rec_year]['DOCS_BEFORE_DATE']/su_data[su_data.YEAR == rec_year]['DOCS_AFTER_DATE']
        perc_diff = perc_diff.values[0]
        perc_diff_prev = 1 - su_data[su_data.YEAR == rec_year - 1]['DOCS_BEFORE_DATE']/su_data[su_data.YEAR == rec_year - 1]['DOCS_AFTER_DATE']
        perc_diff_prev = perc_diff_prev.values[0]
        title = "В {}-м месяце {} года после применения рекомендации <br>кол-во проданных билетов {} на {:.0f} ({:.2f}%)<br>"\
             "по сравнению с предыдущим годом"
        
    fig = px.line(su_data, x='YEAR', y=data_column, 
            title=title.format(month, rec_year, 
                               'увеличилось' if rec_year_diff > 0 else 'уменьшилось',
                               rec_year_diff, (perc_diff - perc_diff_prev)*100))
    fig.add_trace(go.Scatter(x=[rec_year], 
                             y=su_data[su_data.YEAR == rec_year][data_column].values,
                             mode='markers',
                             name = 'Год рекомендации'))
    fig.add_trace(go.Scatter(x=[rec_year - 1], 
                             y=su_data[su_data.YEAR == rec_year - 1][data_column].values,
                             mode='markers',
                             name = 'Предыдущий год'))
    
    fig_trend = px.scatter(su_data, 
                           x='YEAR', y=data_column, 
                           trendline='ols', 
                           color_discrete_sequence =['green'],
                           title='trendline')
    fig.add_trace(fig_trend.data[1], )
    fig.show()
    return fig
    
    
    

def analyse_su_data(data, sale_date, month_num, rec_year, data_column):
    su_plot_data = process_su_data(data, sale_date, month_num)
    plot_sales(su_plot_data, rec_year, data_column)        

In [315]:
analyse_su_data(su_data, '21-03', 6, 2019, 'DOCS_DIFFERENCE')

In [316]:
dds_data = pd.read_csv('data/RIVALS_DATA.csv', sep=';')
dds_data = dds_data[dds_data['DIR'] == 'PARBKK']
#dds_data['SALE_DATE'] = pd.to_datetime(dds_data.LAST_SALE_MONTH*100 + 1 , format='%Y%m%d') + \
#                        pd.DateOffset(months=1) - pd.DateOffset(days=1)
dds_data['SALE_DATE'] = pd.to_datetime(dds_data.LAST_SALE_MONTH*100 + 1 , format='%Y%m%d')
dds_data.head()

,DIR,LAST_SALE_MONTH,TRAVEL_MONTH,PAX_COUNT,SALE_MONTH_NUM,SALE_YEAR,TRAVEL_MONTH_NUM,TRAVEL_YEAR,SALE_DATE
809,PARBKK,202001,202005,1678,1,2020,5,2020,2020-01-01
814,PARBKK,202002,202003,20000,2,2020,3,2020,2020-02-01
818,PARBKK,201412,201503,2693,12,2014,3,2015,2014-12-01
5647,PARBKK,201708,201802,3439,8,2017,2,2018,2017-08-01
5994,PARBKK,201509,201603,504,9,2015,3,2016,2015-09-01


In [317]:
def process_rivals_data(rivals_data, sale_date, month_num):
    _loc_rivals_data = rivals_data[rivals_data['TRAVEL_MONTH_NUM'] == month_num].copy()
    dataset = []
    for year in _loc_rivals_data['SALE_YEAR'].unique():
        year_sale_date = sale_date + '-' + str(year)
        year_rec = []
        year_rec.append(year)
        year_rec.append(month_num)
        
        year_rec.append(_loc_rivals_data[(_loc_rivals_data['SALE_YEAR'] == year)&
                                     (_loc_rivals_data['SALE_DATE'] <= year_sale_date)]['PAX_COUNT'].sum())
        
        year_rec.append(_loc_rivals_data[(_loc_rivals_data['SALE_YEAR'] == year)&
                                     (_loc_rivals_data['SALE_DATE'] > year_sale_date)]['PAX_COUNT'].sum())
        
        dataset.append(year_rec)
        
    data_df = pd.DataFrame(dataset, columns=['YEAR', 'MONTH','DOCS_BEFORE_DATE', 
                                          'DOCS_AFTER_DATE'])
    
    data_df['DOCS_DIFFERENCE'] = data_df['DOCS_AFTER_DATE'] - data_df['DOCS_BEFORE_DATE']
    
    
    data_df.sort_values(by='YEAR', inplace=True)    
    return data_df

In [318]:
fig_su_sales = plot_sales(process_su_data(su_data, '21-03', 6), 2019, data_column = 'SALES_DIFFERENCE')
fig_su_docs = plot_sales(process_su_data(su_data, '21-03', 6), 2019, data_column = 'DOCS_DIFFERENCE')
fig_rivals = plot_sales(process_rivals_data(dds_data, '21-03', 6), 2019, 'DOCS_DIFFERENCE')
